# Place for experimenting the progressive design

In [2]:
import os,sys
import yaml

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
from model_discovery.model.gab_composer import GABTree,ROOT_UNIT_TEMPLATE


ckpt_dir = os.environ['CKPT_DIR']
db_dir = U.pjoin(ckpt_dir, 'test_composer', 'db')
# test_tree = GABTree('test_tree', db_dir)


# prompts_dir='../model_discovery/agents/prompts/'
# dir_prompt = U.pjoin(prompts_dir, 'gu_prompts.yaml')

# with open(dir_prompt, 'r') as file:
#    gu_prompts = yaml.safe_load(file)
# gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
# gam_py = U.read_file(U.pjoin(prompts_dir,'gam_prompt.py'))


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [4]:
ROOT_UNIT_TEMPLATE.replace('{name}', 'test_tree')

'# test_tree.py\n\nimport torch\nimport torch.nn as nn\n\nfrom model_discovery.model.utils.modules import GABUnit # DO NOT CHANGE THIS IMPORT STATEMENT #\n\n\n# YOU CAN IMPORT MORE MODULES HERE #\n\n# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #\n\n\nclass test_tree(GABUnit): \n    """Generalized Autoregressive Block\n        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}\n        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}\n        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable\n    """\n    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #\n        # argv: list of hyperparameters\n        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers\n        super().__init__(embed_

ModuleNotFoundError: No module named 'model_discovery'

In [11]:
print(gu_prompts['GU_designer']['init_design'])

Design a novel autoregressive model block by completing the blanks marked in the Python code file gab.py below, which includes the initialization where you can define your custom arguments, the forward function where you can define convenient functions in the GAB class such as caches, the configuration with the hyperparameters that correspond to the arguments you defined:
```python {gab_py} ```
The GAB is inherited from this GABBase class, you should always import it by "from model_discovery.model.utils.modules import GABBase", you should never remove this statement from gab.py and you should never define another GABBase class in gab.py:
```python {gab_base} ```
This code will be used to construct a gam model in gam.py:
```python {gam_py} ```
This is the configuration and references for the target model:
```python {config} ```
Here are some hints: 1. You need to consider the GAM model structure and the default operations like the normalization when designing the GAB block. Always remem